# Synthesis Experiments

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import matplotlib.patches as pltp
from matplotlib.animation import FuncAnimation, PillowWriter
from numpy import exp, pi, array, asarray
from sympy.solvers import solve
from sympy import Symbol, Ellipse, Point, Line, Matrix, diff


In [3]:
w = 1
t_begin = 0
steps = 100
t_end = np.abs(4 * np.pi / w)

center1 = Point(0, 0)
center2 = Point(0, 2.5)
R1 = 1
R2 = 1.5

d1 = 0.8
phi1 = 0
d2 = 1.4
phi2 = np.pi
act_len = 2

In [4]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
xdata, ydata = [], []
ln, = plt.plot([], [], 'g--')

trace = []
points = []

prismatic_bar = plt.plot([], [], 'r-')
actuator_bar = plt.plot([], [], 'black')

def init_frame():
    ax.set_xlim(-4, 4)
    ax.set_ylim(-2, 7)

    circle1 = pltp.Circle((center1.x, center1.y), R1, linestyle='--', color='b',fill=False)
    circle2 = pltp.Circle((center2.x, center2.y), R2, linestyle='--', color='b', fill=False)
    ax.add_patch(circle1)
    ax.add_patch(circle2)

    plt.plot(center1.x, center1.y, 'bo')
    plt.plot(center2.x, center2.y, 'bo')

    return ln,

# animation update on every frame
def update_frame(frame):
    for i in range(len(points)):
        points[i][0].remove()
    points.clear()

    t = frame
    p1 = Point(center1.x + d1 * np.cos(w * t + phi1), center1.y + d1 * np.sin(w * t + phi1))
    p2 = Point(center2.x + d2 * np.cos(w * t + phi2), center2.y + d2 * np.sin(w * t + phi2))
    dist = np.sqrt((float(p1.x) - float(p2.x)) ** 2 + (float(p1.y) - float(p2.y)) ** 2)
    new_len = dist + act_len
    p3 = Point(new_len / dist * (p2 - p1) + p1)

    points.append(plt.plot(p1.x, p1.y, 'ro'))
    points.append(plt.plot(p2.x, p2.y, 'ro'))
    points.append(plt.plot(p3.x, p3.y, 'go'))

    prismatic_bar[0].set_data([p1.x, p2.x], [p1.y, p2.y])
    actuator_bar[0].set_data([p2.x, p3.x], [p2.y, p3.y])

    trace.append([p3.x, p3.y])
    ln.set_data([x[0] for x in trace], [x[1] for x in trace])
    return ln,

# create animation
anim = FuncAnimation(fig, update_frame, init_func=init_frame, frames=np.linspace(t_begin, t_end, steps), blit=True)

# save animation to the file
anim.save('assets/exp1.gif', dpi=100, writer=PillowWriter(fps=60))
plt.close('all')
# plt.show()
# prevent unclosed plots

In [114]:
def get_act(p1, p2, act_len):
    dist = sp.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)
    new_len = dist + act_len
    p12 = Point(new_len / dist * (p2 - p1) + p1)
    return p12

In [153]:
w1 = 1
w2 = -1
t_begin = 0
steps = 100
t_end = np.abs(4 * np.pi / w)
dt = (t_end - t_begin) / steps
ts = np.linspace(t_begin, t_end, steps)
t = sp.Symbol('t')

center1 = Point(0, 0)
center2 = Point(0, 2.5)
R1 = 1
R2 = 1.5

d1 = 0.8
phi1 = 0
d2 = 1.4
phi2 = 3 * np.pi / 4
act_len1 = 2



center3 = Point(11, 0)
center4 = Point(11, 2.5)
R3 = 1
R4 = 1.5

d3 = 0.8
phi3 = np.pi
d4 = 1.4
phi4 = 0
act_len2 = 2


In [117]:
# Points

rev1 = Point(center1.x + d1 * sp.cos(w1 * t + phi1), center1.y + d1 * sp.sin(w1 * t + phi1))
rev2 = Point(center2.x + d2 * sp.cos(w1 * t + phi2), center2.y + d2 * sp.sin(w1 * t + phi2))
ee1 = get_act(rev1, rev2, act_len1)

rev3 = Point(center3.x + d3 * sp.cos(w2 * t + phi3), center3.y + d3 * sp.sin(w2 * t + phi3))
rev4 = Point(center4.x + d4 * sp.cos(w2 * t + phi4), center4.y + d4 * sp.sin(w2 * t + phi4))
ee2 = get_act(rev3, rev4, act_len2)

In [138]:
# Precalculate all trajectories

rev1s = []
rev2s = []
ee1s = []
rev3s = []
rev4s = []
ee2s = []

for frame in ts:
    rev1s.append(rev1.subs(t, frame))
    rev2s.append(rev2.subs(t, frame))
    ee1s.append(ee1.subs(t, frame))
    rev3s.append(rev3.subs(t, frame))
    rev4s.append(rev4.subs(t, frame))
    ee2s.append(ee2.subs(t, frame))

rev1s = np.array(rev1s, dtype=np.float64)
rev2s = np.array(rev2s, dtype=np.float64)
ee1s = np.array(ee1s, dtype=np.float64)
rev3s = np.array(rev3s, dtype=np.float64)
rev4s = np.array(rev4s, dtype=np.float64)
ee2s = np.array(ee2s, dtype=np.float64)

In [164]:
# Precisely calculate end-effector trajectory
# For better velocity calculation

ee1s_precise = []
ee2s_precise = []
precision_dt = 0.07
precision_steps = int((t_end - t_begin) / precision_dt)

cur_time = 0
for _ in range(precision_steps):
    ee1s_precise.append(ee1.subs(t, cur_time))
    if len(ee1s_precise) > 1 and ee1s_precise[-1] == ee1s_precise[0]:
        break
    cur_time += precision_dt
for _ in range(precision_steps):
    ee2s_precise.append(ee2.subs(t, cur_time))
    if len(ee2s_precise) > 1 and ee2s_precise[-1] == ee2s_precise[0]:
        break
    cur_time += precision_dt

ee1s_precise = np.array(ee1s_precise, dtype=np.float64)
ee2s_precise = np.array(ee2s_precise, dtype=np.float64)

In [168]:
# Find positions for throw as a point where derivative changes its sign

throw1 = []
throw2 = []
maxv1 = 0.0
maxv2 = 0.0
for i in range(1, precision_steps):
    maxv1 = max(maxv1, np.linalg.norm(ee1s_precise[i] - ee1s_precise[i - 1]))
    maxv2 = max(maxv2, np.linalg.norm(ee2s_precise[i] - ee2s_precise[i - 1]))
for i in range(1, precision_steps):
    if np.linalg.norm(ee1s_precise[i] - ee1s_precise[i - 1]) == maxv1:
        throw1 = ee1s_precise[i]
        break
for i in range(1, precision_steps):
    if np.linalg.norm(ee2s_precise[i] - ee2s_precise[i - 1]) == maxv2:
        throw2 = ee2s_precise[i]
        break
print(throw1, throw2)

[0.02807479 3.10725118] [10.6876862   3.07988735]


In [170]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax = np.array(ax)
ax = ax.flatten()
ax[0].set_aspect('equal')
xdata, ydata = [], []
ln, = plt.plot([], [], 'g--')

points = []

prismatic_bar1 = ax[0].plot([], [], 'r-')
actuator_bar1 = ax[0].plot([], [], 'black')

prismatic_bar2 = ax[0].plot([], [], 'r-')
actuator_bar2 = ax[0].plot([], [], 'black')

trace1, = ax[0].plot(ee1s_precise[:, 0], ee1s_precise[:, 1], 'g--')
trace2, = ax[0].plot(ee2s_precise[:, 0], ee2s_precise[:, 1], 'g--')
v1_points = [0.0]
v2_points = [0.0]
vs1, = ax[1].plot([], [], label='Velocity (left)')
vs2, = ax[1].plot([], [], label='Velocity (right)')

for i in range(1, len(ts)):
    v1_points.append(np.linalg.norm(ee1s[i] - ee1s[i - 1]))
    v2_points.append(np.linalg.norm(ee2s[i] - ee2s[i - 1]))

frames = []

alpha1_points = []
alpha2_points = []

alpha1, = ax[2].plot([], [], label='Angle (left)')
alpha2, = ax[2].plot([], [], label='Angle (right)')

def init_frame():
    ax[0].set_xlim(-4, 15)
    ax[0].set_ylim(-2, 10)

    circle1 = pltp.Circle((center1.x, center1.y), R1, linestyle='--', color='b',fill=False)
    circle2 = pltp.Circle((center2.x, center2.y), R2, linestyle='--', color='b', fill=False)

    circle3 = pltp.Circle((center3.x, center3.y), R1, linestyle='--', color='b',fill=False)
    circle4 = pltp.Circle((center4.x, center4.y), R2, linestyle='--', color='b', fill=False)
    ax[0].add_patch(circle1)
    ax[0].add_patch(circle2)

    ax[0].add_patch(circle3)
    ax[0].add_patch(circle4)

    ax[0].plot(center1.x, center1.y, 'bo')
    ax[0].plot(center2.x, center2.y, 'bo')

    ax[0].plot(center3.x, center3.y, 'bo')
    ax[0].plot(center4.x, center4.y, 'bo')

    ax[0].plot(throw1[0], throw1[1], 'go')
    ax[0].plot(throw2[0], throw2[1], 'go')

    ax[0].set_axis_off()

    ax[1].set_xlim(t_begin, t_end)
    ax[1].set_ylim(0, 2)

    ax[1].set_xlabel('t (s)')
    ax[1].set_ylabel('v(t) (m/s)')
    ax[1].grid(True)

    ax[2].set_xlim(t_begin, t_end)
    ax[2].set_ylim(- 1, 2 * np.pi + 1)
    ax[2].set_xlabel('t')
    ax[2].set_ylabel('$\phi({t})$')
    ax[2].grid(True)

    return ln,

# animation update on every frame
def update_frame(frame):
    for i in range(len(points)):
        points[i][0].remove()
    points.clear()

    frames.append(frame)

    # Plot first mechanism

    p1 = rev1s[len(frames) - 1]
    p2 = rev2s[len(frames) - 1]
    p12 = ee1s[len(frames) - 1]

    points.append(ax[0].plot(p1[0], p1[1], 'ro'))
    points.append(ax[0].plot(p2[0], p2[1], 'ro'))
    points.append(ax[0].plot(p12[0], p12[1], 'go'))

    prismatic_bar1[0].set_data([p1[0], p2[0]], [p1[1], p2[1]])
    actuator_bar1[0].set_data([p2[0], p12[0]], [p2[1], p12[1]])

    vs1.set_data([x for x in frames], [x for x in v1_points[:len(frames)]])

    # Calculate angles

    alpha1_points.append(((w1 * frame + phi1) % (2 * np.pi)) * np.sign(w1))
    alpha1.set_data(frames, alpha1_points)

    p3 = rev3s[len(frames) - 1]
    p4 = rev4s[len(frames) - 1]
    p34 = ee2s[len(frames) - 1]

    points.append(ax[0].plot(p3[0], p3[1], 'ro'))
    points.append(ax[0].plot(p4[0], p4[1], 'ro'))
    points.append(ax[0].plot(p34[0], p34[1], 'go'))

    prismatic_bar2[0].set_data([p3[0], p4[0]], [p3[1], p4[1]])
    actuator_bar2[0].set_data([p4[0], p34[0]], [p4[1], p34[1]])

    vs2.set_data([x for x in frames], [x for x in v2_points[:len(frames)]])

    # Calculate angles

    alpha2_points.append(((w2 * frame + phi3) % (2 * np.pi)))
    alpha2.set_data(frames, alpha2_points)

    return ln,


ax[1].legend()
ax[2].legend()
# create animation
anim = FuncAnimation(fig, update_frame, init_func=init_frame, frames=ts, blit=True)

# save animation to the file
anim.save('assets/exp2.gif', dpi=100, writer=PillowWriter(fps=60))
plt.close('all')
# plt.show()
# prevent unclosed plots